In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import torch
import random
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
import logging
import cv2

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

logging.getLogger('albumentations').setLevel(logging.WARNING)
from albumentations.pytorch import ToTensorV2

from rsna2024.runner import Runner
from rsna2024.preproc.generate_tiles import get_tile
from rsna2024 import model as module_model
from rsna2024.utils import (
    rsna_lumbar_metric,
    natural_sort,
    sagi_coord_to_axi_instance_number,
    get_series,
)

coord_filename = 'train_label_coordinates_predicted_{}_{}_{}.csv'.format(593, 654, 603)

root_dir = '/media/latlab/MR/projects/kaggle-rsna-2024'
data_dir = os.path.join(root_dir, 'data', 'raw')
img_dir = os.path.join(data_dir, 'train_images')
df_series = pd.read_csv(
    os.path.join(data_dir, 'train_series_descriptions.csv'),
    dtype={'study_id': 'str', 'series_id': 'str'},
)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

levels = ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']
sides = ['left', 'right']
coord_df = pd.read_csv(
    os.path.join(root_dir, 'data', 'processed', coord_filename),
    dtype={'study_id': 'str', 'series_id': 'str'}
)


def load_config(config_path):
    with open(config_path) as f:
        return json.load(f)


def get_metric(y_true, y_pred):
    solution, submission = rsna_lumbar_metric.prepare_data(y_true, y_pred)
    metric = rsna_lumbar_metric.score(
        solution=solution,
        submission=submission,
        row_id_column_name='row_id',
        any_severe_scalar=1.0,
    )
    return metric

def print_loss(orig_baseline_loss, preds, ys):
    baseline_loss = CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 4.0]).to(device))(torch.tensor(preds).to(device), torch.tensor(ys).to(device))
    print(f'\nLoss: {orig_baseline_loss:.4f} -> {baseline_loss:.4f}')
    print(f'Improvement: {orig_baseline_loss - baseline_loss:.4f}, {100 * (orig_baseline_loss - baseline_loss) / orig_baseline_loss:.1f}%')

## ROI Models
### Spinal

In [ ]:
spinal_model_name = 'vague-glitter-719' #'vague-glitter-719' 0.2909 'comic-snowball-706' # 0.2918 # fine-field-600 0.3063
cfg = load_config(os.path.join(root_dir, 'models', 'rsna-2024-' + spinal_model_name, 'config.json'))
spinal_preds, spinal_ys, spinal_data = Runner(
    cfg, model_name='rsna-2024-' + spinal_model_name
).predict(df_coordinates=coord_df)

# unfolding 5 cv folds and 5 levels
spinal_preds = np.moveaxis(spinal_preds.reshape(5, 5, -1, 3), 1, -1).reshape(-1, 3, 5)
spinal_ys = np.moveaxis(spinal_ys.reshape(5, 5, -1), 1, -1).reshape(-1, 5)

print_loss(0.2909, spinal_preds, spinal_ys)

### Foraminal

In [ ]:
foraminal_model_name = 'eager-brook-732' #'eager-brook-732' #0.5272 # winter-thunder-708# 0.5272  #helpful-bush-662 0.5422 #'exalted-rain-631' 0.5551
cfg = load_config(os.path.join(root_dir, 'models', 'rsna-2024-' + foraminal_model_name, 'config.json'))
foraminal_preds, foraminal_ys, foraminal_data = Runner(
    cfg, model_name='rsna-2024-' + foraminal_model_name
).predict(df_coordinates=coord_df)

# unfolding 5 cv folds, 5 levels, (study_ids) and 2 sides
foraminal_preds = np.moveaxis(np.moveaxis(foraminal_preds.reshape(5, 5, -1, 2, 3), 3, -1), 1, -1).reshape(-1, 3, 10)
foraminal_ys = np.moveaxis(np.moveaxis(foraminal_ys.reshape(5, 5, -1, 2), 3, -1), 1, -1).reshape(-1, 10)

print_loss(0.5272, foraminal_preds, foraminal_ys)

### Subarticular

In [ ]:
subarticular_model_name = 'icy-lake-742' #'icy-lake-742' 0.5978 # 'efficient-puddle-714' 0.6056 #'bumbling-meadow-608' 0.6334 flowing-haze-618
cfg = load_config(os.path.join(root_dir, 'models', 'rsna-2024-' + subarticular_model_name, 'config.json'))
subarticular_preds, subarticular_ys, subarticular_data = Runner(
    cfg, model_name='rsna-2024-' + subarticular_model_name
).predict(df_coordinates=coord_df)

# unfolding 5 cv folds, 5 levels and 2 sides
subarticular_preds = np.moveaxis(np.moveaxis(subarticular_preds.reshape(5, 5, -1, 2, 3), 3, -1), 1, -1).reshape(-1, 3, 10)
subarticular_ys = np.moveaxis(np.moveaxis(subarticular_ys.reshape(5, 5, -1, 2), 3, -1), 1, -1).reshape(-1, 10)

print_loss(0.5978, subarticular_preds, subarticular_ys)

### Global improvement

In [ ]:
preds = torch.tensor(np.concatenate([spinal_preds, foraminal_preds, subarticular_preds], axis=-1)).to(device)
ys = torch.tensor(np.concatenate([spinal_ys, foraminal_ys, subarticular_ys], axis=-1)).to(device)

baseline_metric = 0.4267
metric = get_metric(ys, preds)
print(f'Baseline metric: {baseline_metric:.4f} -> {metric:.4f}')
print(f'Improvement: {baseline_metric - metric:.4f}, {100 * (baseline_metric - metric) / baseline_metric:.1f}%')